In [1]:
import numpy as np
from datetime import datetime
import os

### Just random testing stuff

In [11]:
now = datetime.now().strftime("%Y-%m-%d_%Hh-%Mm-%Ss")
print(now)

outf = os.path.join(os.getcwd(), 'figures', 'sim_'+now+'.mp4')
print(outf)

2023-09-23_12h-14m-33s
/home/janmak98/Documents/masterthesis/chemokine/monte_carlo/figures/sim_2023-09-23_12h-14m-33s.mp4


In [101]:

particle_diffusivity = [1, 2, 3,4, 5]
friction = np.array(particle_diffusivity)**0.7
# print(friction)
# print(particle_diffusivity)
# print(np.array(particle_diffusivity))
# uno = np.array([1,1,1], ndmin=2).T
# print(uno)
# prob = np.append(uno, friction.T, axis=1)
# prob = np.append(prob, friction.T, axis=1)
# print(prob)
# print(np.array([np.array([1,1,1], ndmin=2).T, np.array(friction, ndmin=2).T, friction]).T)
probabilities = np.array([np.array([1,1,1,1,1]), np.array(friction), np.array(friction)]).T
#probabilities = np.array([[1,1,1], particle_diffusivity, particle_diffusivity]).T
print(probabilities)
probabilities = probabilities/np.sum(probabilities, axis=1, keepdims=True)

print(probabilities)
step_choice = np.array([0, 1, -1])
n_particles_0 = 10

step = np.random.choice(step_choice, size=(n_particles_0, 2), p=probabilities[0])

print(probabilities[0])
# print(probabilities[1].shape)
# print(np.sum(probabilities[1]))
print(step)

[[1.         1.         1.        ]
 [1.         1.62450479 1.62450479]
 [1.         2.15766928 2.15766928]
 [1.         2.63901582 2.63901582]
 [1.         3.08516931 3.08516931]]
[[0.33333333 0.33333333 0.33333333]
 [0.23534896 0.38232552 0.38232552]
 [0.18813477 0.40593261 0.40593261]
 [0.15928559 0.4203572  0.4203572 ]
 [0.13946343 0.43026829 0.43026829]]
[0.33333333 0.33333333 0.33333333]
[[ 0  1]
 [ 1  0]
 [-1  1]
 [ 1 -1]
 [-1  0]
 [ 1  1]
 [ 1  0]
 [ 0 -1]
 [ 1  0]
 [ 1  0]]


In [1]:
pos = np.random.randint(0, 10, size=(10, 2))

print(pos)
collagen_y = 5

idx_fixed = np.where(pos[:, 1] == collagen_y)[0]
print(idx_fixed)

print(pos[idx_fixed])
print(idx_fixed.shape[0])

NameError: name 'np' is not defined

In [ ]:
grid_size = [10, 12]
n_particles_3 = 5

pos_3x = np.random.randint(0, grid_size[0], size=(n_particles_3))
pos_3y = np.random.randint(0, grid_size[1], size=(n_particles_3))
pos_3 = np.array([pos_3x, pos_3y]).T
print(pos_3)

[[ 9  1]
 [ 0  7]
 [ 2 11]
 [ 9  5]
 [ 6  6]]


In [6]:
a = 1 
b = 2

if a==1 and b==2:
    print('yes')
elif a==1:
    print('1')
elif b==2:
    print('2')
else:
    print('no')

yes


# Compute hopping probabilities with neighbor particle present

In [4]:
# from physical_units_diffusion import *

In [15]:

# the indexing of the concentrations, diffusivities and energies has to be always in the same order, i.e. concentrations=[c1, c2, ...] then diffusivitiy_array=[D1, D2, ...]
# function that takesin the matrix containing all binding energies in kbT and diffusivities in [m^2/s] and outputs the ratio of time step to grid constant squared Delta/a^2
def calc_time_step_ratio(energy_matrix, diffusivity_array):
    # compute ratio of time step to grid constant squared Delta/a^2 by setting the maximum movement probability to be 1/4
    if len(diffusivity_array) != len(energy_matrix):
        print("Error: length of diffusivity array must be the same as the number of rows in the energy matrix")
        return
    max_dexp = 0 # initialize maximum product of diffusivity with exp(energy)
    for i in range(len(energy_matrix)):
        for j in range(i + 1, len(energy_matrix)):
            if np.exp(energy_matrix[i, j]) * diffusivity_array[j] > max_dexp:
                max_dexp = np.exp(energy_matrix[i, j]) * diffusivity_array[j] # maximum of occuring product

    max_d = np.max(diffusivity_array)
    max_tot = np.max([max_dexp, max_d]) # if all binding energies are negative, moving probability is higher for pure diffusion
    ratio = 1 / (5 * max_tot)
    return ratio

# function that takes  thye ratio of time step to grid constant squared Delta/a^2 in [s/m^2] and a diffusivity in [m^2/s] and returns the probability of a particle to move in one direction and the probability to stay in the same position
def calc_probabilities_free(ratio, diffusivity_array):
    p_move = diffusivity_array * ratio * 1.25
    p_stay = 1 - 4 * p_move
    return p_move, p_stay

# function computes moving probability in one direction with a neighbor present in this direction
def calc_probabilities_with_neighbor(energy_matrix, diffusivity_array):
    ratio = calc_time_step_ratio(energy_matrix, diffusivity_array)
    p_move_free = calc_probabilities_free(ratio, diffusivity_array)[0]
    p_move_neighbor = np.zeros(int(len(energy_matrix) * (len(energy_matrix) - 1) / 2))
    # one could make a matrix in the shape of energy_matrix, but i+j-1 is also unique
    for i in range(len(energy_matrix)):
        for j in range(i + 1, len(energy_matrix)):
            p_move_neighbor[i + j - 1] = p_move_free[i] * np.exp(energy_matrix[i, j])
            # print(i + j - 1)
    return p_move_neighbor



# function taking concentrations of particle species in [mol/l], wanted number of rarest species and ratio Delta/a^2 in [s/m^2] and outputs the grid constant a in [m], the time step in [s] and the number of all species
def calc_nspecies_gridconst(ratio, concentrations, n_min, grid_dim):
    a_grid = (n_min / ((grid_dim[0] - 1) * (grid_dim[1] - 1) * np.min(concentrations) * 6.022e26)) ** (1/3)
    n_species = np.int64(n_min * concentrations / np.min(concentrations))
    time_step = ratio * a_grid**2
    return n_species, a_grid, time_step


# estimate diffusivity from mass of molecule in [kDa] in case no experimental diffusivity in water is given
# input can be a numpy array or float
def estimate_diffusivity(mass):
    # approximate hydrodynamic radius of protein by average protein density of 1.38 g/cm^3 = 1.38 * 1e3 kg/m^3
    # for 10kDa protein this gives a radius of 1.42 nm and at 300K a diffusion coefficient of 1.5e-10 m^2/s
    mass_kg = mass * 1.66054e-24 #for given mas in [kDa] calculate mass in [kg]
    r = (3 * mass_kg / (1.38 * 1e3 * 4 * np.pi)) ** (1/3) # calculate radius in [m]
    # use stokes law D=kBT/gamma with gamma=6*pi*eta*r and water viscosity ~ 1e-3 Pa*s to calculate diffusivity in [m^2/s]
    D = 1.38e-23 * 300 / (6 * np.pi * 1e-3 * r) 
    return D

# return concentration in [mol/l] for given number of n_species particles in given grid
# assumes volume of (n-1)a x (m-1)a x a in simulation slab
def physical_concentration(n_species, grid_dim, a_grid):
    concentration = n_species / ((grid_dim[0] - 1) * (grid_dim[1] - 1) * a_grid**3 * 6.022e26)
    return concentration

## Use random values in the right order of magnitude to get a feeling for jumping probabilities, grid size and time step

In [16]:
energy_matrix = np.random.rand(25) * 6 + 4 # between 4-10
energy_matrix = energy_matrix.reshape((5, 5))
energy_matrix += energy_matrix.T # symmetric
energy_matrix -= np.diagonal(energy_matrix) # no selfbinding
diffusivity_array = estimate_diffusivity(np.array([50, 23, 200, 1000, 100]))

ratio = calc_time_step_ratio(energy_matrix, diffusivity_array)
print('probs', calc_probabilities_free(ratio, diffusivity_array))
print('probs neighbors', calc_probabilities_with_neighbor(energy_matrix, diffusivity_array), np.max(calc_probabilities_with_neighbor(energy_matrix, diffusivity_array)))

concentrations = np.array([70, 10, 300, 100, 20]) * 1e-9 #random concentrations in nanomolar
n_min = 100
grid_dim = (100, 100)
n_species, a_grid, time_step = calc_nspecies_gridconst(ratio, concentrations, n_min, grid_dim)
print(n_species, a_grid, time_step)

print(physical_concentration(n_species, grid_dim, a_grid)) # reproduces input concentrations

probs (array([1.50328852e-04, 1.94740552e-04, 9.47012423e-05, 5.53816225e-05,
       1.19316089e-04]), array([0.99939868, 0.99922104, 0.9996212 , 0.99977847, 0.99952274]))
probs neighbors [1.92986066e-01 1.33921091e-05 6.79934305e-05 2.37958242e-02
 1.15715208e-04 9.35419411e-04 1.86515612e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[ 700  100 3000 1000  200] 1.1921464774740315e-07 1.8916592075333236e-08
[7.e-08 1.e-08 3.e-07 1.e-07 2.e-08]
